In [28]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.stats import ttest_rel
from utils.evaluate import Evaluate 
%matplotlib inline

In [69]:
def get_all_LTR_metrics(file, ids, metrics):
    results = []
    for idx in ids:
        df = pd.read_pickle(file.format(idx))
        for column in df:
            if type(column) is int:
                # Get a ranking with the respective scores.
                ranking = df[str(column) + "_s"].as_matrix()
                
                # Remove all nans
                ranking = [int(x) for x in ranking if not math.isnan(float(x))]
                
                # Calculate the evaluation scores
                scores = Evaluate.compute_scores(ranking)
                
                # Append all evaluation scores together with its query id in the correct order
                results.append([column] + [scores[i] for i in metrics])
                
    df = pd.DataFrame(results, columns=["query_id"] + metrics)
    return df.groupby("query_id", as_index=False).mean().sort_values(by=["query_id"])

def t_test(df_1, df_2, metrics):
    results = []
    for metric in metrics:
        results.append((metric, ttest_rel(df_1[metric], df_2[metric]).pvalue))
    return results
    

In [81]:
pickle_path = "storage/logs"
files = ["_baseline_masks_{}.pkl",
         "_ViP_masks_{}.pkl",
        "_ViP_snapshots_{}.pkl",
        "_ViP_highlights_{}.pkl",
        "_vgg16_snapshots_{}.pkl",
        "_vgg16_highlights_{}.pkl",
        "_vgg16_saliency_{}.pkl"]
metrics = ["p@1","p@5","p@10","ndcg@1","ndcg@5","ndcg@10","map"]

ids = range(1, 26)#[1, 6, 11, 16, 21]
dfs = []
for file in files: 
    dfs.append(get_all_LTR_metrics(os.path.join(pickle_path, file), ids, metrics))
    


In [91]:
t_test(dfs[0], dfs[4], metrics)

[('p@1', 8.76228458199429e-06),
 ('p@5', 4.069809580260542e-06),
 ('p@10', 4.899520872865728e-07),
 ('ndcg@1', 0.00014228953257556043),
 ('ndcg@5', 1.953976796259634e-05),
 ('ndcg@10', 1.854361372616378e-06),
 ('map', 0.0024153486644061892)]

In [92]:
dfs[6]

,query_id,p@1,p@5,p@10,ndcg@1,ndcg@5,ndcg@10,map
0,201,0.2,0.68,0.78,0.100000,0.315768,0.367129,0.677616
1,202,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.012461
2,203,1.0,0.32,0.44,0.866667,0.330018,0.338038,0.433338
3,204,1.0,1.00,0.96,0.600000,0.821207,0.813079,0.612883
4,205,0.4,0.44,0.44,0.133333,0.171928,0.203945,0.302681
5,206,0.6,0.52,0.46,0.300000,0.289854,0.296066,0.427826
6,207,1.0,0.72,0.72,0.333333,0.335929,0.388195,0.585504
7,208,0.4,0.40,0.22,0.133333,0.171507,0.136476,0.240249
8,209,0.0,0.00,0.06,0.000000,0.000000,0.033848,0.098344
9,210,0.0,0.28,0.28,0.000000,0.131068,0.157586,0.178383


In [95]:
dfs[0]

,query_id,p@1,p@5,p@10,ndcg@1,ndcg@5,ndcg@10,map
0,201,0.6,0.56,0.56,0.300000,0.298270,0.307955,0.679989
1,202,0.2,0.04,0.02,0.200000,0.200000,0.200000,0.209454
2,203,0.6,0.48,0.54,0.200000,0.236438,0.258308,0.462866
3,204,0.0,0.16,0.20,0.000000,0.090913,0.112573,0.389261
4,205,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.156002
5,206,0.8,0.44,0.56,0.400000,0.246009,0.315381,0.474720
6,207,0.6,0.52,0.48,0.333333,0.297050,0.311283,0.502974
7,208,0.0,0.16,0.20,0.000000,0.054636,0.077778,0.241439
8,209,0.2,0.12,0.08,0.133333,0.120572,0.098024,0.127290
9,210,0.0,0.08,0.06,0.000000,0.049242,0.045388,0.135749
